In [71]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [72]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [73]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long).to(device)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)].to(device)
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1]).to(device)
    return (max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))).to(device)

In [74]:
# from torchtext.vocab import GloVe

#         # 指定GloVe的维度和版本
# glove = GloVe(name='6B', dim=50)

In [75]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        
        # embedding_matrix = glove.get_vecs_by_tokens(list(word_to_ix.keys()))

        # embedding_matrix = glove.get_vecs_by_tokens(word_to_ix.keys())
        # self.word_embeds = nn.Embedding.from_pretrained(embedding_matrix).to(device)
        
        

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim).to(device)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True).to(device)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size).to(device)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size)).to(device)

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas
        forward_var = forward_var.to(device)

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                emit_score = emit_score.to(device)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                trans_score = trans_score.to(device)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags.to(device)]).to(device)
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]].to(device)
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]].to(device)
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars.to(device)
        
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag].to(device)
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1).to(device)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]].to(device)
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence).to(device)
        forward_score = self._forward_alg(feats).to(device)
        gold_score = self._score_sentence(feats, tags).to(device)
        return (forward_score - gold_score).to(device)

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [76]:
# tensor = glove.get_vecs_by_tokens(['', '1998', '199999998', ',', 'cat'], True)
# print(tensor)

In [77]:
# START_TAG = "<START>"
# STOP_TAG = "<STOP>"
# EMBEDDING_DIM = 5
# HIDDEN_DIM = 4

# # Make up some training data
# training_data = [(
#     "the wall street journal reported today that apple corporation made money".split(),
#     "B I I I O O O B I O O".split()
# ), (
#     "georgia tech is a university in georgia".split(),
#     "B I O O O O B".split()
# )]

# word_to_ix = {}
# for sentence, tags in training_data:
#     for word in sentence:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

# tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

# model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
# optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# # Check predictions before training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
#     precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
#     print(model(precheck_sent))

# # Make sure prepare_sequence from earlier in the LSTM section is loaded
# for epoch in range(
#         300):  # again, normally you would NOT do 300 epochs, it is toy data
#     for sentence, tags in training_data:
#         # Step 1. Remember that Pytorch accumulates gradients.
#         # We need to clear them out before each instance
#         model.zero_grad()

#         # Step 2. Get our inputs ready for the network, that is,
#         # turn them into Tensors of word indices.
#         sentence_in = prepare_sequence(sentence, word_to_ix)
#         targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

#         # Step 3. Run our forward pass.
#         loss = model.neg_log_likelihood(sentence_in, targets)

#         # Step 4. Compute the loss, gradients, and update the parameters by
#         # calling optimizer.step()
#         loss.backward()
#         optimizer.step()

# # Check predictions after training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
#     print(model(precheck_sent))
# # We got it!

In [78]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

def addbatch(data_train,data_test,batchsize):
    """
    设置batch
    :param data_train: 输入
    :param data_test: 标签
    :param batchsize: 一个batch大小
    :return: 设置好batch的数据集
    """
    data = TensorDataset(data_train,data_test)
    data_loader = DataLoader(data, batch_size=batchsize, shuffle=False)#shuffle是是否打乱数据集，可自行设置

    return data_loader

In [79]:
import numpy as np
a = np.arange(3)
b = np.arange(3)[:, np.newaxis]
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

In [82]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
import pandas as pd  

# data = pd.read_csv('train.txt', sep="\t", header=None)
# data.columns = ["word", "type"]

# df = pd.DataFrame(data) 
# training_data=[]
# tmp = []

# for col in df.columns:
#     l = df[col].to_list()
#     tmp.append(l)
    
# training_data.append(tuple(tmp))

with open('train.txt') as f:
    contents = f.read()
    data = contents.split('\n\n')
     
result = []

for block in data:
    lines = block.strip().split('\n')
    if len(lines[0]) == 0: 
        break
    words, tags = zip(*[line.split('\t') for line in lines])
    result.append((list(words), list(tags)))

training_data = result

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
# traindata=addbatch(training_data,trainlabel,1000)#1000为一个batch_size大小为1000，训练集为10000时一个epoch会训练10次。


tag_to_ix = {"O": 0, "B-geo-loc": 1, "B-facility": 2, "I-facility": 3, "B-movie": 4, 
             "I-movie": 5, "B-company": 6, "B-product": 7, "B-person": 8, "B-sportsteam": 9,
             "I-sportsteam": 10, "I-product": 11, "B-other": 12, "I-other": 13, "I-company": 14,
             "I-person": 15, "I-geo-loc": 16, "B-tvshow": 17, "B-musicartist": 18, "I-musicartist": 19,
             "I-tvshow": 20, START_TAG: 21, STOP_TAG: 22}

ix_to_tag = {v: k for k, v in tag_to_ix.items()}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

from tqdm import tqdm

# Check predictions before training
output = []
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix).to(device)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long).to(device)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(100):
    total_loss = 0.0  # 用于累积损失

    # 使用tqdm包装训练数据迭代，并设置描述
    for sentence, tags in tqdm(training_data, desc=f"Epoch {epoch}"):
        # Step 1. Remember that PyTorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix).to(device)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets).to(device)

        # 累积损失
        total_loss += loss.item()

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

    # 打印平均损失
    average_loss = total_loss / len(training_data)
    print(f"Epoch {epoch}, Average Loss: {average_loss:.4f}")

from sklearn.metrics import f1_score
# Check predictions after training
output = []
with torch.no_grad():
    for i in range(100):
        precheck_sent = prepare_sequence(training_data[i][0], word_to_ix).to(device)
        output = [ix_to_tag[p] for p in model(precheck_sent)[1]]
        print(output)
        print(training_data[i][1])
        print(f1_score(output, training_data[i][1], average='micro'))


# We got it!

(tensor(27.2051, device='cuda:0'), [11, 14, 3, 3, 3, 3, 3, 3, 3, 11, 14, 4])


Epoch 0: 100%|██████████| 2394/2394 [12:57<00:00,  3.08it/s]


Epoch 0, Average Loss: 7.3696


Epoch 1: 100%|██████████| 2394/2394 [08:33<00:00,  4.67it/s]


Epoch 1, Average Loss: 6.9471


...

Epoch 97, Average Loss: 2.7967


Epoch 98: 100%|██████████| 2394/2394 [03:32<00:00, 11.29it/s]


Epoch 98, Average Loss: 2.7933


Epoch 99: 100%|██████████| 2394/2394 [03:32<00:00, 11.29it/s]


Epoch 99, Average Loss: 2.7811
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
1.0
['O', 'O', 'O', 'O', 'O', 'B-geo-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'B-geo-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-facility', 'I-facility', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-facility', 'I-facility', 'O', 'O']
0.8787878787878788
['O', 'B-other', 'B-person', 'O', 'O', 'O', 'B-geo-loc', 'O', 'O', 'O', 'O']
['O', 'B-movie', 'I-movie', 'O', 'O', 'O', 'B-geo-loc', 'O', 'O', 'O', 'O']
0.8181818181818182
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [81]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))


NVIDIA GeForce RTX 3090
